<center><img src="https://storage.googleapis.com/arize-assets/arize-logo-white.jpg" width="200"/></center>

# Predictive Maintenance in the Manufacturing Industry
Walk through how to use Arize for a predictive maintenance model using an example dataset.

## 📨 Upload Data to Arize
Upload example data to Arize, this example uses the [Python Pandas method](https://docs.arize.com/arize/sending-data-methods/log-directly-via-sdk-api).

In [ ]:
# Install and import dependencies

!pip install -q arize
from arize.pandas.logger import Client, Schema
from arize.utils.types import ModelTypes, Environments, Metrics

import pandas as pd
import numpy as np
import datetime

### 🌐 Upload Data to Arize: Download Data
Here are sample parquet files that represent the <strong>training</strong> and <strong>production</strong> data of a model designed to predict machine maintenance based on features such as:
* Machine Age: Age of the machine in months
* Environment: Physical environment that includes factors such as humidity, dust levels, ambient temperature, etc.
* Parts Replaced: If the machine has had repairs or maintenance issues
* Pressure: Measured in PSI to indicate blockages
* Operational Time: Maximum time the machine is in use

In [ ]:
production_df = pd.read_parquet(
    "https://storage.googleapis.com/arize-assets/fixtures/Industry_Use_Case/manufacturing_predictive_maintenance_production.parquet",
)
training_df = pd.read_parquet(
    "https://storage.googleapis.com/arize-assets/fixtures/Industry_Use_Case/manufacturing_predictive_maintenance_training.parquet",
)

In [ ]:
#create a prediction timestamp column

production_df.drop('prediction_ts', axis=1, inplace=True)

current_time = datetime.datetime.now().timestamp()

earlier_time = (
    datetime.datetime.now() - datetime.timedelta(days=30)
).timestamp()

optional_prediction_timestamps = np.linspace(
    earlier_time, current_time, num=production_df.shape[0]
)

production_df.insert(1, "prediction_ts", optional_prediction_timestamps.astype(int))
production_df[["prediction_ts"]].head()

### 🤝 Upload Data to Arize: Create Arize Client
Sign up/login to your Arize account <a href="https://app.arize.com/auth/login">here</a>. Find your <a href="https://docs.arize.com/arize/api-reference/arize.pandas/client">Space and API keys</a>. Copy/paste into the cell below.

In [ ]:
SPACE_KEY = "YOUR_SPACE_KEY"  # update value here with your Space Key
API_KEY = "YOUR_API_KEY"  # update value here with your API key

arize_client = Client(space_key=SPACE_KEY, api_key=API_KEY)

In [ ]:
if SPACE_KEY == "SPACE_KEY" or API_KEY == "API_KEY":
    raise ValueError("❌ NEED TO CHANGE SPACE AND/OR API_KEY")
else:
    print(
        "✅ Import and Setup Arize Client Done! Now we can start using Arize!"
    )

### 📋 Upload Data to Arize: Define Schema
Create your <a href="https://docs.arize.com/arize/sending-data-to-arize/model-schema-reference">model schema</a>. First, we'll define the features, shap values, and tags.  

In [ ]:
feature_column_names = [ "Machine_Age", "Environment", "Parts_Replaced", "Pressure", "Operational_Time", "Usage"]
shap_column_names = [f"{x}_shap" for x in feature_column_names]

In [ ]:
# Define a Schema() object for Arize to pick up data from the correct columns for logging
training_schema = Schema(
    prediction_id_column_name="prediction_id",
    prediction_label_column_name="Prediction_Labels",
    prediction_score_column_name="Prediction_Scores",
    actual_label_column_name="Actual_Labels",
    actual_score_column_name="Actual_Scores",
    feature_column_names=feature_column_names,
)

# Logging Training DataFrame
training_response = arize_client.log(
    dataframe=training_df,
    model_id="preictive-maintenance-manufacturing",
    model_version="v1.0",
    model_type=ModelTypes.BINARY_CLASSIFICATION,
    environment=Environments.TRAINING,
    schema=training_schema,
)

### 🪵 Log Production Data to Arize

In [ ]:
production_schema = Schema(
    prediction_id_column_name="prediction_id",
    timestamp_column_name="prediction_ts",
    prediction_label_column_name="Prediction_Labels",
    prediction_score_column_name="Prediction_Scores",
    actual_label_column_name="Actual_Labels",
    actual_score_column_name="Actual_Scores",
    feature_column_names=feature_column_names,
    shap_values_column_names=dict(zip(feature_column_names, shap_column_names)),
)

production_response = arize_client.log(
    dataframe=production_df,
    model_id="preictive-maintenance-manufacturing",
    model_version="v1.0",
    model_type=ModelTypes.BINARY_CLASSIFICATION,
    environment=Environments.PRODUCTION,
    schema=production_schema,
)

## 🏃 Follow 'Success!' Link To Arize
Once you've successfully logged your model to Arize, follow the link to setup monitors, uncover problem areas, and more!

<strong>Note</strong>: It might take a few minutes for all the data to index in Arize, if you don't see all the data immedieatly, sit back and relax, data is on it's way!

### 🔍 In Arize: Model Setup
It looks like our model is in good shape! Since we uploaded prediction and actual data, we can measure model performance. In this case, our average accuracy is 0.91 right now. Let set up some monitors to keep it that way!

* Naviage to the 'Config' tab to configure your positive class to 'Maintain'.
* Within the 'Config' tab, select 'Configure Baseline', and select 'Pre-Production'

<img src="https://storage.googleapis.com/arize-assets/fixtures/Industry_Use_Case/manufacturing_predictive_maintenance_homepage.png" width="1200">

### 🤖 In Arize: Monitor Setup

Since our model is behaving as expected, we want to configure monitors to get alerted right when our model deviates from expected behavior.
* Navigate to the 'Monitors' tab, select 'Setup Monitors'
* Enable 'False Negative' monitor, and any others that interest you!

<img src="https://storage.googleapis.com/arize-assets/fixtures/Industry_Use_Case/manufacturing_predictive_maintenance_monitors.png" width="1200">

### 🤖 In Arize: Monitor Config

All monitors created from the 'Setup Monitors' tab are configured with default settings. Edit settings to change how sensitive your monitors and where you get alerted.
* Click into the 'Config' tab on the 'Monitors' page

Learn more about monitor settings [here](https://docs.arize.com/arize/monitors/performance-monitors).

<img src="https://storage.googleapis.com/arize-assets/fixtures/Industry_Use_Case/manufacturing_predictive_maintenance_config.png" width="1200">

### 📈 In Arize: Performance Tracing
Lets explore our model performance using the performance tracing tab to identify areas to improve. Since we uploaded traning and production data, we can compare the two to see how our production model.

* Click 'Add Comparison' in the 'Performance Tracing' tab
* Next to 'Dataset B', click on the the drop down menu and select training

<img src="https://storage.googleapis.com/arize-assets/fixtures/Industry_Use_Case/manufacturing_predictive_maintenance_performance.png" width="1200">

### 📈 In Arize: Performance Tracing

Notice how our production model performance dips slightly as compared to training? While slight deviations are expected, let's ingestigate further.

* Scroll down to the 'Performance Insights' card
* Click on 'Operational Time', try excluding the slice with the filter function

Our model performance improves! This indicates to us that our model is heavily influenced by the 'Operational Time' feature.

* Scroll down to the 'Environment' feature.

Notice how there's a missing bar in the training histogram? Click 'View Feature Details' to gain more insights. It looks like our training model is missing some data!

<img src="https://storage.googleapis.com/arize-assets/fixtures/Industry_Use_Case/manufacturing_predictive_maintenance_training.png" width="1200">

## 🚀 Continue Exploring Arize
By visually investigating model performance, we can easily identify problematic areas within our model to consider during the feature engineering and model rebuilding process.

This tutorial just scratches the surface of what Arize can do. Continue to explore the world of ML Observability with Arize to monitor, troubleshoot, and fine tune your models!

<strong>Recommended Resources:</strong>
* [Arize Community Slack](https://join.slack.com/t/arize-ai/shared_invite/zt-1is2wp3xv-SQgwwszCEeS06Sm1q4xFFw)
* [Arize Documentation](https://docs.arize.com/arize/)
* [ML Observability Course](https://courses.arize.com/)